In [85]:
# %load "C:/Users/wangj337/Google Drive/Courses/IntroAI-CS50/2Uncertainty/pagerank/pagerank.py"
import os
import random
import re
import sys
from collections import Counter
import numpy

DAMPING = 0.85
SAMPLES = 10000


def main():
    if len(sys.argv) != 2:
        sys.exit("Usage: python pagerank.py corpus")
    corpus = crawl(sys.argv[1])
    ranks = sample_pagerank(corpus, DAMPING, SAMPLES)
    print(f"PageRank Results from Sampling (n = {SAMPLES})")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")
    ranks = iterate_pagerank(corpus, DAMPING)
    print(f"PageRank Results from Iteration")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")


def crawl(directory):
    """
    Parse a directory of HTML pages and check for links to other pages.
    Return a dictionary where each key is a page, and values are
    a list of all other pages in the corpus that are linked to by the page.
    """
    # The keys in that dictionary represent pages (e.g., "2.html"), and the values of the dictionary are a set of all of the pages linked to by the key (e.g. {"1.html", "3.html"}).
    pages = dict()

    # Extract all links from HTML files
    for filename in os.listdir(directory):
        if not filename.endswith(".html"):
            continue
        with open(os.path.join(directory, filename)) as f:
            contents = f.read()
            links = re.findall(r"<a\s+(?:[^>]*?)href=\"([^\"]*)\"", contents)
            pages[filename] = set(links) - {filename}

    # Only include links to other pages in the corpus
    for filename in pages:
        pages[filename] = set(
            link for link in pages[filename]
            if link in pages
            # If there is a link to wiki in page 1, but wiki is not in the corpse folder
            # then wiki will not be part of page 1's links. 
            # "We only rank what we know for sure.", said Jin.
        )

    return pages



In [181]:
def transition_model(corpus, page, damping_factor):
    """
    Return a probability distribution over which page to visit next,
    given a current page.

    With probability `damping_factor`, choose a link at random
    linked to by `page`. With probability `1 - damping_factor`, choose
    a link at random chosen from all pages in the corpus.
    """
    #print(corpus)
    result = dict()
    N = len(corpus)
    N_page = len(corpus[page])
    # ignore the link to its own page
    if page in corpus[page]:
        N_page -= 1

    # With prob = 1 - damping_factor, randomly choose from all pages
    for p in corpus:
        result[p] = 1 / N * (1 - damping_factor)
        
    # With prob = damping_factor, randomly choose from links
    for p in corpus[page]:
        if p == page:
            continue
        else:
            result[p] += 1 / N_page * damping_factor
        
    # Normalize for floating errors:
    factor = 1.0 / sum(result.values())
    for p in result:
        result[p] = result[p]*factor

    # The return value of the function should be a Python dictionary with one 
    # key for each page in the corpus. Each key should be mapped to a value 
    # representing the probability that a random surfer would choose that page next. 
    # The values in this returned probability distribution should sum to 1
    return result


In [210]:
def sample_pagerank(corpus, damping_factor, n):
    """
    Return PageRank values for each page by sampling `n` pages
    according to transition model, starting with a page at random.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    
    samples = list()
    
    # First sample: choosing from a page at random.
    current = random.choice(list(corpus.keys()))
    
    # Next sample: generated from the previous sample based on the 
    # previous sample’s transition model.
    for i in range(n):
        trans = transition_model(corpus, current, damping_factor)
        current = random.choices(
            population = list(trans.keys()),
            weights = trans.values(),
            k = 1
        )[0]
        samples.append(current)     
        
    # Summary of samples    
    frequency = Counter(samples)
    for page in frequency:
        frequency[page] = frequency[page] / n      
    return frequency 


def iterate_pagerank(corpus, damping_factor):
    """
    Return PageRank values for each page by iteratively updating
    PageRank values until convergence.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    
    # Preprocessing
    N = len(corpus)
    numLinks = numpy.array([len(corpus[p]) for p in corpus], dtype=float)
    numLinks[numLinks == 0] = N
    prob = numpy.reciprocal(numLinks)
    binaryLinks = numpy.zeros((N, N))
    for i, p1 in enumerate(corpus.keys()):
        if not corpus[p1]:
        # A page that has no links at all should be interpreted 
        # as having one link for every page in the corpus (including itself).
            for j in range(N):
                binaryLinks[j, i] = 1
        else:
            for j, p2 in enumerate(corpus.keys()):
                if p2 in corpus[p1]:
                    # ignore the link to its own
                    binaryLinks[j, i] = 1
    
    # Start: 1 / N for every page
    current = numpy.ones(N) / N
    #print(current)
    
    # Repeatedly update rank values until no change in values > 0.001.
    difference = 1
    while difference > 0.001: 
        new = (1 - damping_factor) * numpy.ones(N) / N + \
          (damping_factor) * numpy.sum(binaryLinks * prob * current, axis = 1)
        difference = max(abs(current - new))
        current = new
    
    result = dict()
    for i, page in enumerate(corpus):
        result[page] = new[i]
    return result


In [ ]:
if __name__ == "__main__":
    main()

In [236]:
%run pagerank.py "corpus0"

PageRank Results from Sampling (n = 10000)
  1.html: 0.2201
  2.html: 0.4291
  3.html: 0.2214
  4.html: 0.1294
PageRank Results from Iteration
  1.html: 0.2198
  2.html: 0.4294
  3.html: 0.2198
  4.html: 0.1311


In [237]:
%run pagerank.py "corpus1"

PageRank Results from Sampling (n = 10000)
  bfs.html: 0.1165
  dfs.html: 0.0774
  games.html: 0.2287
  minesweeper.html: 0.1162
  minimax.html: 0.1308
  search.html: 0.2126
  tictactoe.html: 0.1178
PageRank Results from Iteration
  bfs.html: 0.1152
  dfs.html: 0.0809
  games.html: 0.2277
  minesweeper.html: 0.1180
  minimax.html: 0.1312
  search.html: 0.2090
  tictactoe.html: 0.1180


In [238]:
%run pagerank.py "corpus2"

PageRank Results from Sampling (n = 10000)
  ai.html: 0.1931
  algorithms.html: 0.1056
  c.html: 0.1214
  inference.html: 0.1333
  logic.html: 0.0280
  programming.html: 0.2258
  python.html: 0.1193
  recursion.html: 0.0735
PageRank Results from Iteration
  ai.html: 0.1889
  algorithms.html: 0.1064
  c.html: 0.1238
  inference.html: 0.1289
  logic.html: 0.0264
  programming.html: 0.2301
  python.html: 0.1238
  recursion.html: 0.0717


In [213]:
iterate_pagerank(toy, 0.8)

{'1.html': 0.4543820027434842,
 '2.html': 0.4543820027434842,
 '3.html': 0.09123599451303153}

In [214]:
toy = {'1.html': {'2.html'}, 
       '2.html': {'1.html', '2.html'}, 
       '3.html': set()}
sample_pagerank(toy, 0.8, 10000)

Counter({'1.html': 0.4504, '2.html': 0.4558, '3.html': 0.0938})

In [205]:
%cd C:/Users/wangj337/Google Drive/Courses/IntroAI-CS50/2Uncertainty/pagerank/
corpus = crawl("corpus0")
print(corpus)
print(len(corpus))
print(len(corpus['2.html']))
test = transition_model(corpus, '1.html', 0.6)
sum(test.values())
#test

C:\Users\wangj337\Google Drive\Courses\IntroAI-CS50\2Uncertainty\pagerank
{'1.html': {'2.html'}, '2.html': {'3.html', '1.html'}, '3.html': {'4.html', '2.html'}, '4.html': {'2.html'}}
4
2


0.9999999999999999

In [235]:
    corpus['1.html'] = set()
    corpus['4.html'] = {'4.html', '2.html'}

    N = len(corpus)
    numLinks = numpy.array([len(corpus[p]) for p in corpus], dtype=float)\
      - numpy.array([(p in corpus[p]) for p in corpus], dtype=float)
    numLinks[numLinks == 0] = N
    prob = numpy.reciprocal(numLinks)
    binaryLinks = numpy.zeros((N, N))
    for i, p1 in enumerate(corpus.keys()):
        if not corpus[p1]:
            for j in range(N):
                binaryLinks[j, i] = 1
        else:
            for j, p2 in enumerate(corpus.keys()):
                if p2 in corpus[p1] and p1 != p2:
                    # ignore the link to its own
                    binaryLinks[j, i] = 1

print(corpus)                    
print(binaryLinks)
#numpy.sum(binaryLinks, axis = 0)
numLinks

{'1.html': set(), '2.html': {'3.html', '1.html'}, '3.html': {'4.html', '2.html'}, '4.html': {'4.html', '2.html'}}
[[1. 1. 0. 0.]
 [1. 0. 1. 1.]
 [1. 1. 0. 0.]
 [1. 0. 1. 0.]]


array([4., 2., 2., 1.])

In [120]:
a = numpy.array([len(corpus[p]) for p in corpus], dtype=float)
print(a)
print(numpy.reciprocal(a))
b = numpy.array([0, 1, 2, 3], dtype=float)
a[a == 1] = 100
print(a)
print(b)
print(numpy.inner(a, b))
print(abs(b - a))
max(abs(b - a))

[1. 2. 2. 1.]
[1.  0.5 0.5 1. ]
[100.   2.   2. 100.]
[0. 1. 2. 3.]
306.0
[100.   1.   0.  97.]


100.0

In [87]:
N = len(corpus)
current = numpy.ones(N) / N
current

array([0.25, 0.25, 0.25, 0.25])

In [38]:
print(test)
keys = random.sample(test.keys(), 1)
keys[0]

{'1.html': 0.1, '2.html': 0.7, '3.html': 0.1, '4.html': 0.1}


<ipython-input-38-5dfdde2a5716>:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  keys = random.sample(test.keys(), 1)


'3.html'

In [37]:
test.keys()

dict_keys(['1.html', '2.html', '3.html', '4.html'])

In [59]:
test.values()

dict_values([0.1, 0.7, 0.1, 0.1])

In [69]:
mylist = ["apple", "banana", "cherry"]
print(random.choice(mylist))

res = random.choices(
    population=list(test.keys()),
    weights=test.values(),
    k=100
)
frequency = Counter(res)

for p in frequency:
    frequency[p] = frequency[p] / 100

frequency

cherry


Counter({'4.html': 0.15, '2.html': 0.71, '1.html': 0.08, '3.html': 0.06})

In [61]:
result = list()
    
current = random.choice(list(test.keys()))
result.append(current)
result

['1.html']

In [58]:
result[-1]

'3.html'

In [215]:
!jupyter nbconvert --to script "scratch.ipynb" --output pagerank

[NbConvertApp] Converting notebook scratch.ipynb to script
[NbConvertApp] Writing 8326 bytes to pagerank.py
